## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('borrowers.csv')  # Серверный путь

print(df.shape) #оценим размер данных
df.info() #оценим распределение типов данных по столбцам и наличие пропусков в информации
df.head(10) #выведем первые 10 строк для предварительного анализа данных

(21525, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
print('Уникальные значения столбца children:', *df['children'].unique(), sep='|')

Уникальные значения столбца children:|1|0|3|2|-1|4|20|5


In [5]:
print('Уникальные значения столбца dob_years:', *df['dob_years'].sort_values().unique(), sep='|')

Уникальные значения столбца dob_years:|0|19|20|21|22|23|24|25|26|27|28|29|30|31|32|33|34|35|36|37|38|39|40|41|42|43|44|45|46|47|48|49|50|51|52|53|54|55|56|57|58|59|60|61|62|63|64|65|66|67|68|69|70|71|72|73|74|75


In [6]:
print('Уникальные значения столбца education:', *df['education'].str.lower().unique(), sep='|')

Уникальные значения столбца education:|высшее|среднее|неоконченное высшее|начальное|ученая степень


In [7]:
print('Уникальные значения столбца family_status:', *df['family_status'].unique(), sep='|')

Уникальные значения столбца family_status:|женат / замужем|гражданский брак|вдовец / вдова|в разводе|Не женат / не замужем


In [8]:
print('Уникальные значения столбца gender:', *df['gender'].unique(), sep='|')

Уникальные значения столбца gender:|F|M|XNA


In [9]:
print('Уникальные значения столбца income_type:', *df['income_type'].unique(), sep='|')

Уникальные значения столбца income_type:|сотрудник|пенсионер|компаньон|госслужащий|безработный|предприниматель|студент|в декрете


In [10]:
print('Уникальные значения столбца debt:', *df['debt'].unique(), sep='|')

Уникальные значения столбца debt:|0|1


In [11]:
print(df[df['total_income'] < 0]['total_income'].count()) #проверим нет ли отрицательного дохода у клиентов

0


In [12]:
print('Уникальные значения столбца purpose:', *df['purpose'].unique(), sep='|')

Уникальные значения столбца purpose:|покупка жилья|приобретение автомобиля|дополнительное образование|сыграть свадьбу|операции с жильем|образование|на проведение свадьбы|покупка жилья для семьи|покупка недвижимости|покупка коммерческой недвижимости|покупка жилой недвижимости|строительство собственной недвижимости|недвижимость|строительство недвижимости|на покупку подержанного автомобиля|на покупку своего автомобиля|операции с коммерческой недвижимостью|строительство жилой недвижимости|жилье|операции со своей недвижимостью|автомобили|заняться образованием|сделка с подержанным автомобилем|получение образования|автомобиль|свадьба|получение дополнительного образования|покупка своего жилья|операции с недвижимостью|получение высшего образования|свой автомобиль|сделка с автомобилем|профильное образование|высшее образование|покупка жилья для сдачи|на покупку автомобиля|ремонт жилью|заняться высшим образованием


In [13]:
df.duplicated().sum() #оценим наличие дублей

54

### Выявленные проблемы на этапе изучения общей информации:

1. Отрицательный стаж в столбце days_employed
2. Слишком большой стаж в столбце days_employed. Пример: 340266.072047
3. Дни в столбце days_employed имеют тип float64 и содержат знаки после запятой.
4. В 2174 строчках в столбцах days_employed и total_income отсутствуют данные.
5. Замечен разный регистр в столбце education.
6. В столбце children имеется значение -1.
7. Для ряда клиентов в столбце dob_years установлен возраст "0".
8. В столбце gender найдено значение XNA. Скорей всего это характерная замена типу None, т.к. серфинг на яндексе не дал результатов отнесения данного сокращения к полу человека.
9. В столбце purpose выделено много похожих целей, например: "покупка недвижимости" и "покупка коммерческой недвижимости". Именно для обработки этого столбца необходимо будет воспользоваться лемматизацией.
10. Имеется 54 дубликата.

### Шаг 2. Предобработка данных

### Обработка пропусков

В шаге 1 выявлены пропуски в столбцах days_employed и total_income. Посмотрим примеры данных с пропущенными значениями для принятия решения, что с ними делать.

In [14]:
df[df['total_income'].isnull()].info() #проверим, что значение NaN для столбцов days_employed и total_income находятся в одних
#и тех же строках и нет необходимости анализировать их по отдельности
df[df['total_income'].isnull()].head(5) #посмотрим данные

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          2174 non-null   int64  
 1   days_employed     0 non-null      float64
 2   dob_years         2174 non-null   int64  
 3   education         2174 non-null   object 
 4   education_id      2174 non-null   int64  
 5   family_status     2174 non-null   object 
 6   family_status_id  2174 non-null   int64  
 7   gender            2174 non-null   object 
 8   income_type       2174 non-null   object 
 9   debt              2174 non-null   int64  
 10  total_income      0 non-null      float64
 11  purpose           2174 non-null   object 
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Из представленных данных видно, что несмотря на незаполненные данные в столбцах days_employed и total_income, во всех остальных столбцах данные заполнены. Предположительно, это связано с нежеланием людей указывать свой реальный стаж и уровень заработка, т.к. это может повлиять на решение по выдаче им кредита.
Данные в столбце days_employed не нужны для ответов на вопросы, поставленные заказчиком, поэтому пропуски в этом столбце будут заменены значением 0.

In [15]:
df['days_employed'] = df['days_employed'].fillna(0) #заменим значения NaN на 0 в столбце days_employed
df.info() #проверим что пропуски заменены удачно

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Данные в столбце income_type понадобятся для ответов на вопросы, поставленные заказчиком, поэтому пропуски должны быть заполнены средними значениями. При этом необходимо определить каким подходом пользоваться: записью среднего арифмитического либо медианы. В любом случае оценка этих значений должна проходить в разрезе типов занятости, потому что не логично сравнивать зарплату компаньена и стипендию студента. Для определения что нужно применять оценим предельные значения дохода для каждого типа занятости, среднее арифметическое и медианные значения.

In [16]:
unique_income_types = df['income_type'].unique() #создадим список всех типов занятости
for type in unique_income_types: #для каждого типа занятости посмотрим минимальный и максимальный доход
    print('Минимальный доход для типа занятости', type, 'составляет', df[df['income_type'] == type]['total_income'].min())
    print('Максимальный доход для типа занятости', type, 'составляет', df[df['income_type'] == type]['total_income'].max())
    print('Среднее арифмитическое дохода для типа занятости', type, 'составляет', df[df['income_type'] == type]['total_income'].mean())
    print('Медианное значение дохода для типа занятости', type, 'составляет', df[df['income_type'] == type]['total_income'].median())
    print()

Минимальный доход для типа занятости сотрудник составляет 21367.64835648697
Максимальный доход для типа занятости сотрудник составляет 1726276.0143316735
Среднее арифмитическое дохода для типа занятости сотрудник составляет 161380.26048788553
Медианное значение дохода для типа занятости сотрудник составляет 142594.39684740017

Минимальный доход для типа занятости пенсионер составляет 20667.26379327158
Максимальный доход для типа занятости пенсионер составляет 735103.2701666558
Среднее арифмитическое дохода для типа занятости пенсионер составляет 137127.4656901654
Медианное значение дохода для типа занятости пенсионер составляет 118514.48641164352

Минимальный доход для типа занятости компаньон составляет 28702.812888839853
Максимальный доход для типа занятости компаньон составляет 2265604.028722744
Среднее арифмитическое дохода для типа занятости компаньон составляет 202417.4614617771
Медианное значение дохода для типа занятости компаньон составляет 172357.95096577113

Минимальный дохо

Анализ показателей дохода в разрезе типов занятости показал, что значения среднего арифметического и медианы либо равны либо незначительно отличаются, поэтому для заполнения пропусков может быть принят любой из двух подходов. Для заполнения пропусков в столбце total_income будем вносить средние арифмитические значения, соответствующие своим типам занятости.

In [17]:
df[df['income_type'] == 'пенсионер'].head(3) #проверку выполним на типе занятости пенсионер. в 12 строке сейчас NaN

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
12,0,0.000000,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля


In [18]:
for type in unique_income_types: #создадим цикл по заполнению пропусков в столбце total_income в разрезе каждого типа занятости
    mean = df[df['income_type'] == type]['total_income'].mean()
    df.loc[(df['income_type'] == type) & (df['total_income'].isnull()), 'total_income'] = mean
df.info() #проверим, что нулевых значений не осталось

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [19]:
df[df['income_type'] == 'пенсионер'].head(3) #в 12 строке в столбце total_income теперь среднее значение для типа пенсионер

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
12,0,0.000000,65,среднее,1,гражданский брак,1,M,пенсионер,0,137127.465690,сыграть свадьбу
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля


### Вывод

Значения NaN в столбце days_employed заменены на 0, т.к. они не требуются нам для ответов на вопросы заказчика.

Значения NaN в столбце total_income заменены средними артфметическими значениями для каждого типа занятости в отдельности.

Вызов метода info() показал, что значений NaN больше нет в датафрейме, задачу 1 шага 2 считаем завершенной.

### Замена типа данных

In [20]:
print(df['days_employed'].sum()) #проверим точно ли все данные в столбце days_employed имеют тип float
print(df['total_income'].sum()) #проверим точно ли все данные в столбце total_income имеют тип float
# В случае наличия данных другого типа при выполнении кода возникнет ошибка

1220012776.2471714
3603197086.325668


Суммирование значений в столбцах прошло без ошибок, следовательно во всех строках данных столбцов находятся данные типа float64 и преобразования в данный тип из какого-либо другого не требуется. Для преобразования в вещественный тип достаточно будет применить метод astype() c аргументом int. Столбец days_employed переводится в вещественный тип данных, т.к. трудовой стаж не требуется вести с точностью до часа (или точнее). Столбец total_income переводится в вещественный тип данных, т.к. точности до рубля должно хватать для оценки платежеспосбности клиента. Проведем замену вещественного типа данных на целочисленный для данных 2-х столбцов.

In [21]:
df['days_employed'] = df['days_employed'].astype('int') #проводим преобразование float в int для столбца days_employed
df['total_income'] = df['total_income'].astype('int') #проводим преобразование float в int для столбца total_income
df.info() #проверяем результат преобразования

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int32 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int32 
 11  purpose           21525 non-null  object
dtypes: int32(2), int64(5), object(5)
memory usage: 1.8+ MB


### Вывод

Вызов метода info() показал, что в обрабатываемом датафрейме отсутствуют данные вещественного типа. Ошибок при преобразовании не возникло. Задачу 2 шага 2 считаем завершенной. 

### Обработка дубликатов

Перед обработкой дубликатов наиболее корректным будет провести обработку выявленных артефактов, влияющих на подготовку ответов на вопросы заказчика. На этапе изучения общей информации, представленной в таблице, были выявлены следующие ошибки и артефакты:

1. Отрицательный стаж в столбце days_employed.
2. Слишком большой стаж в столбце days_employed. Пример: 340266.072047
3. Замечен разный регистр в столбце education.
4. Для ряда клиентов в столбце dob_years установлен возраст "0".
5. В столбце gender найдено значение XNA.
6. В столбце children имеется значение -1.

*Рассмотрим выявленные артефакты и ошибки.*

1. Обработаем отрицательный стаж в столбце days_employed.

In [22]:
print(df[df['days_employed'] < 0]['days_employed'].count()) #посчитаем количество строк с отрицательным стажем
df[df['days_employed'] < 0].head(5) #выведем несколько строк для ознакомления

15906


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
5,0,-926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья


Для 74 % строк датафрейма введен отрицательный стаж, что говорит о системной ошибке, которую нельзя исключить путем замены на среднее значение или что-то подобное. Предположительно, часть анкет, заполняемых клиентами банка, имела следующий формат внесения данных о трудовом стаже: "Ваш трудовой стаж - " и при распознавании их системой CRM тире было считано как минус. Поэтому для таких строк требуется произвести исключение минуса перед числом.

In [23]:
df['days_employed'] = df['days_employed'].abs() #перезаписываем значения столбца по модулю
df[df['days_employed'] < 0]['days_employed'].count() #производим проверку на наличие отрицательных значений

0

2. Оценим артефакт со слишком большим стажем

In [24]:
df['dob_years'].max() #найдем максимальный возраст клиента

75

In [25]:
print(df[df['days_employed'] > 22265]['days_employed'].count()) #примерно посчитаем количество человек с некорректным стажем,
# исходя из того, что максимальный возраст = 75 лет, офиц. труд стаж может пойти с 14 лет, тогда макс кол-во раб. дней составляет 22265
df[df['days_employed'] > 22265].sort_values('days_employed').head(5) #посмотрим несколько таких строк

3445


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
20444,0,328728,72,среднее,1,вдовец / вдова,2,F,пенсионер,0,96519,покупка жилья для семьи
9328,2,328734,41,высшее,0,женат / замужем,0,M,пенсионер,0,126997,операции со своей недвижимостью
17782,0,328771,56,среднее,1,женат / замужем,0,F,пенсионер,0,68648,операции с коммерческой недвижимостью
14783,0,328795,62,высшее,0,женат / замужем,0,F,пенсионер,0,79940,на покупку своего автомобиля
7229,1,328827,32,среднее,1,гражданский брак,1,F,пенсионер,0,122162,сыграть свадьбу


In [26]:
#в выведенных выше данных появились только строчки с пенсионерами, проверим кто вообще попадает в этот артефакт
df[df['days_employed'] > 22265]['income_type'].value_counts() 

пенсионер      3443
безработный       2
Name: income_type, dtype: int64

Оценка данного артефакта показала, что в основном для типа занятости "пенсионер" добавляется дополнительных 800-900 лет стажа. Данный артефакт не влияет на решение задачи заказчика, поэтому не обязателен к обработке. Необходимо сообщить заказчику о неверных данных.

3. Приведем значения в столбце education к единому регистру

In [27]:
print('Уникальные значения столбца education:', *df['education'].unique(), sep='|') #начальная проверка значений
df['education'] = df['education'].str.lower() #изменение регистра
print('Уникальные значения столбца education:', *df['education'].unique(), sep='|') #проверка значений после изменения

Уникальные значения столбца education:|высшее|среднее|Среднее|СРЕДНЕЕ|ВЫСШЕЕ|неоконченное высшее|начальное|Высшее|НЕОКОНЧЕННОЕ ВЫСШЕЕ|Неоконченное высшее|НАЧАЛЬНОЕ|Начальное|Ученая степень|УЧЕНАЯ СТЕПЕНЬ|ученая степень
Уникальные значения столбца education:|высшее|среднее|неоконченное высшее|начальное|ученая степень


4. Оценим количество клиентов с возрастом 0

In [28]:
print(df[df['dob_years'] == 0]['dob_years'].count()) #посчитаем количество клиентов с 0-м возрастом
df[df['dob_years'] == 0].head() # посмотрим что это за клиенты

101


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541,0,среднее,1,женат / замужем,0,F,пенсионер,0,71291,автомобиль
149,0,2664,0,среднее,1,в разводе,3,F,сотрудник,0,70176,операции с жильем
270,3,1872,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166,ремонт жилью
578,0,397856,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620,строительство собственной недвижимости
1040,0,1158,0,высшее,0,в разводе,3,F,компаньон,0,303994,свой автомобиль


Из приведенной выборки видно, что клиенты с 0-м возрастом совершенно разные люди. Но схожи они в одном - не заполнили графу возраст при подаче своих данных. Т.к. этот артефакт не влияет на решение нашей задачи, данные в датафрейме оставляем и уведомляем о их наличии заказчика.

5. Проведем анализ клиентов с полом XNA

In [29]:
df[df['gender'] == 'XNA'] # посмотрим что это за клиенты

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905,покупка недвижимости


Всего 1 клиент не определился со своим полом (или система CRM банка не смогла распознать его). Посмотрим с каким полом клиентов больше и присвоим этот пол нашему 1-му клиенту.

In [30]:
if (df[df['gender'] == 'F']['gender'].count()) >= (df[df['gender'] == 'M']['gender'].count()):
    df.loc[df['gender'] == 'XNA', 'gender'] = 'F'
else:
    df.loc[df['gender'] == 'XNA', 'gender'] = 'M'
df['gender'].unique() #посмотрим остался ли артефакт

array(['F', 'M'], dtype=object)

Артефакт XNA заменен.

6. Обработаем значения "-1" в столбце children.

In [31]:
print(df[df['children'] == -1]['children'].count()) #посчитаем количество таких клиентов
df[df['children'] == -1].head(5) #выведем несколько строк для оценки данных

47


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
291,-1,4417,46,среднее,1,гражданский брак,1,F,сотрудник,0,102816,профильное образование
705,-1,902,50,среднее,1,женат / замужем,0,F,госслужащий,0,137882,приобретение автомобиля
742,-1,3174,57,среднее,1,женат / замужем,0,F,сотрудник,0,64268,дополнительное образование
800,-1,349987,54,среднее,1,Не женат / не замужем,4,F,пенсионер,0,86293,дополнительное образование
941,-1,0,57,среднее,1,женат / замужем,0,F,пенсионер,0,137127,на покупку своего автомобиля


Предположительно, как и в случае с отрицательным стажем, знак тире из анкеты был распознан в качестве минуса. Преобразуем -1 в 1.

In [32]:
df['children'] = df['children'].abs() #перезаписываем значения столбца по модулю
df['children'].unique() #проверим, что значений -1 больше нет

array([ 1,  0,  3,  2,  4, 20,  5], dtype=int64)

Также в выборке замечено значение 20. Посмотрим сколько таких клиентов.

In [33]:
print(df[df['children'] == 20]['children'].count())

76


Согласно статистическим данным 2018 года в России имелось всего 943 семьи с количеством детей 11 и более. Вероятность того, что 76 из этих семей являются клиентами нашего банка очень мала, поэтому считаем, что допущена ошибка в обработке анкет и вместо значения "2" было прочитано значение "20". Произведем замену.

Для дополнительной проверки о данном артефакте необходимо сообщить заказчику.

In [34]:
df.loc[df['children'] == 20, 'children'] = 2 #произведем замену 20 на 2
df['children'].unique() # проверим, что значений 20 больше нет

array([1, 0, 3, 2, 4, 5], dtype=int64)

Все артефакты и ошибки обработаны, переходим к обработке дубликатов. Т.к. считаем, что необходимые данные заполнены теперь корректно, можно избавиться от полных дубликатов методом drop_duplicates с последующим применением метода reset_index с удалением старых индексов. Метод dropna не применяем, т.к. значения na в датафрейме уже отсутствуют.

In [35]:
print(df.duplicated().sum()) #оценим количество дубликатов
df = df.drop_duplicates().reset_index(drop=True) #удаляем дубликаты
print(df.duplicated().sum()) #оценим количество дубликатов после удаления

71
0


### Вывод

Перед удалением дубликатов была проведена предварительная обработка датафрейма, в результате которой количество выявленных дубликатов увеличилось с 54 до 71. Все дубликаты были удалены методом drop_duplicates(). Предположительно, появление дубликатов вызвано неоднократным заполнением анкет клиентами (например, заявка на кредит изначально была подана в офисе банка, а потом через мобильное приложение). Задачу 3 шага 2 считаем завершенной.

### Лемматизация

In [36]:
from pymystem3 import Mystem #подключаем библиотеку для лемматизиации
m = Mystem()
print('Уникальные значения столбца purpose:', *df['purpose'].unique(), sep='|') #выведем уникальные цели получения кредита

Уникальные значения столбца purpose:|покупка жилья|приобретение автомобиля|дополнительное образование|сыграть свадьбу|операции с жильем|образование|на проведение свадьбы|покупка жилья для семьи|покупка недвижимости|покупка коммерческой недвижимости|покупка жилой недвижимости|строительство собственной недвижимости|недвижимость|строительство недвижимости|на покупку подержанного автомобиля|на покупку своего автомобиля|операции с коммерческой недвижимостью|строительство жилой недвижимости|жилье|операции со своей недвижимостью|автомобили|заняться образованием|сделка с подержанным автомобилем|получение образования|автомобиль|свадьба|получение дополнительного образования|покупка своего жилья|операции с недвижимостью|получение высшего образования|свой автомобиль|сделка с автомобилем|профильное образование|высшее образование|покупка жилья для сдачи|на покупку автомобиля|ремонт жилью|заняться высшим образованием


In [37]:
purposes_type = ['жилье', 'автомобиль', 'образование', 'свадьба', 'недвижимость'] # выделим основные категории целей
purposes_dict = {} # создадим словарь для хранения соответствия цель - категория цели
unique_purposes = df['purpose'].unique()
for purpose in unique_purposes: # для каждой цели определим категорию и добавим в словарь
    lemma_purpose = m.lemmatize(purpose)
    for lemma in lemma_purpose:
        if lemma in purposes_type:
            purposes_dict[purpose] = lemma
print(purposes_dict) # проверим получившийся словарь

{'покупка жилья': 'жилье', 'приобретение автомобиля': 'автомобиль', 'дополнительное образование': 'образование', 'сыграть свадьбу': 'свадьба', 'операции с жильем': 'жилье', 'образование': 'образование', 'на проведение свадьбы': 'свадьба', 'покупка жилья для семьи': 'жилье', 'покупка недвижимости': 'недвижимость', 'покупка коммерческой недвижимости': 'недвижимость', 'покупка жилой недвижимости': 'недвижимость', 'строительство собственной недвижимости': 'недвижимость', 'недвижимость': 'недвижимость', 'строительство недвижимости': 'недвижимость', 'на покупку подержанного автомобиля': 'автомобиль', 'на покупку своего автомобиля': 'автомобиль', 'операции с коммерческой недвижимостью': 'недвижимость', 'строительство жилой недвижимости': 'недвижимость', 'жилье': 'жилье', 'операции со своей недвижимостью': 'недвижимость', 'автомобили': 'автомобиль', 'заняться образованием': 'образование', 'сделка с подержанным автомобилем': 'автомобиль', 'получение образования': 'образование', 'автомобиль': 'а

In [38]:
def purpose_type_install(purpose): # создадим функцию для назначения категории цели в зависимости от самой цели
    return purposes_dict[purpose]
print(purpose_type_install('профильное образование')) # проверим работу функции

образование


In [39]:
df['purpose_type'] = df['purpose'].apply(purpose_type_install) # применим созданную функцию для создания столбца с категорией цели
df.head() #выведем датафрейм для проверки

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_type
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


### Вывод

Согласно 5 строкам выведенного датафрейма лемматизация прошла успешно - в таблице появился новый столбец, который характеризует категорию цели. Изначально для решения данной задачи возникает желание делать лемматизацию внутри функции для всех значений столбца, но процесс расчета при этом затягивается на несколько часов. Поэтому применение словаря для уникальных значений столбца purpose оказывается в данном случае наиболее приемлемым. Задачу 4 шага 2 считаем завершенной.

### Категоризация данных

Будем категоризировать только те данные датафрейма, которые понадобятся для ответов на поставленные вопросы.
1. Основной фактор всех вопросов - это возврат кредита в срок, значит для более удобного представления данных в датафрейме должен появиться столбец payer_type с возможными значениями: должник, исправный плательщик.
2. Для выявления зависимости от наличия детей добавим в таблицу столбец children_availability со значениями: есть дети, нет детей.
3. В далеком 2017-м госкомстат разделил всех людей по уровням дохода, сейчас данное деление заблокировано на сайте. Но воспользуемся им за основу категоризации данных по уровням дохода с учетом официальных темпов инфляции. Выделим следующие категории: крайняя нищета - 7.8 - 9 т.р., нищета - 9-13.5 т.р, бедность - 13.5-22.5 т.р., выше бедности - 22.5-33.5, средний достаток - 33.5-67.2 т.р., состоятельные - 67.2-100.8 т.р., богатые - 100.8-168 т.р., сверхбогатые > 168 т.р.
4. Категоризация по типам целей кредита сделана в предыдущем шаге.
5. Семейное положение будем рассматривать каждое в отдельности.

1. Создадим категорию payer_type для разделения всех клиентов на должников и исправных плательщиков 

In [40]:
def client_to_payer (debt): # создадим функцию для присвоения категории плательщика клиенту
    if debt == 0:
        return 'исправный плательщик'
    elif debt == 1:
        return'должник'
    else:
        return 'некорректное значение debt'
df['payer_type'] = df['debt'].apply(client_to_payer) # создадим новый столбец с категорией плательщика
print(df.head(3)) #проверим на первых трех строках
df[df['debt'] == 1].head(3) #проверим на первых трех строках с задолженностью

   children  days_employed  dob_years education  education_id  \
0         1           8437         42    высшее             0   
1         1           4024         36   среднее             1   
2         0           5623         33   среднее             1   

     family_status  family_status_id gender income_type  debt  total_income  \
0  женат / замужем                 0      F   сотрудник     0        253875   
1  женат / замужем                 0      F   сотрудник     0        112080   
2  женат / замужем                 0      M   сотрудник     0        145885   

                   purpose purpose_type            payer_type  
0            покупка жилья        жилье  исправный плательщик  
1  приобретение автомобиля   автомобиль  исправный плательщик  
2            покупка жилья        жилье  исправный плательщик  


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_type,payer_type
14,0,1844,56,высшее,0,гражданский брак,1,F,компаньон,1,165127,покупка жилой недвижимости,недвижимость,должник
32,0,4649,34,среднее,1,гражданский брак,1,F,сотрудник,1,139057,на проведение свадьбы,свадьба,должник
38,0,597,25,высшее,0,Не женат / не замужем,4,M,сотрудник,1,192247,образование,образование,должник


2. Добавим столбец children_availability, который будет сообщать есть ли у клиента дети.

In [41]:
def children_have (children): # создадим ф-ю для определения наличия детей
    if children == 0:
        return 'детей нет'
    return 'дети есть'
print(children_have(0)) #проверим работу ф-ии
df['children_availability'] = df['children'].apply(children_have) # создаем новый столбец, говорящий о наличии детей
df.head() # проверяем работу

детей нет


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_type,payer_type,children_availability
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье,исправный плательщик,дети есть
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,исправный плательщик,дети есть
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье,исправный плательщик,детей нет
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,исправный плательщик,дети есть
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,исправный плательщик,детей нет


3. Разобьем всех клиентов на категории по уровню дохода.

In [42]:
def income_level (income): # создаем ф-ю для категоризации уровня дохода
    if 0 <= income < 9000:
        return 'крайняя нищета'
    if 9000 <= income < 13500:
        return 'нищета'
    if 13500 <= income < 22500:
        return 'бедность'
    if 22500 <= income < 33500:
        return 'выше бедности'
    if 33500 <= income < 67200:
        return 'средний достаток'
    if 67200 <= income < 100800:
        return 'состоятельные'
    if 100800 <= income < 168000:
        return 'богатые'
    if 168000 <= income:
        return 'сверхбогатые'
print(income_level(56232)) #проверка ф-ии
df['income_level'] = df['total_income'].apply(income_level) #создаем новый столбец с категорией уровня дохода
df.head() # проверяем работу

средний достаток


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_type,payer_type,children_availability,income_level
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье,исправный плательщик,дети есть,сверхбогатые
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,исправный плательщик,дети есть,богатые
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье,исправный плательщик,детей нет,богатые
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,исправный плательщик,дети есть,сверхбогатые
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,исправный плательщик,детей нет,богатые


### Вывод

Все необходимые категории для подготовки ответов на вопросы заказчика подготовлены. Задачу 4 шага 2 считаем выполненной.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Для подготовки ответов на вопросы заказчика будем пользоваться сводными таблицами pandas. Для определения зависимости возврата кредита в срок от наличия детей создадим сводную таблицу, в которой строками (столбец, по которым групппируют данные) будут категории "детей нет" и "дети есть", столбцами для группировки при этом будут категории плательщика ("должник" или "исправный плательщик").

После создания сводной таблицы оценим процент должников в для каждой из рассматриваемых категорий по наличию детей.

In [43]:
# создаем сводную талбицу по наличию детей
children_pivot = df.pivot_table(index='children_availability', columns='payer_type', values='purpose_type', aggfunc='count' )
# добавляем в сводную процент должников по каждой из категорий по детям
children_pivot['debtor_percent'] =children_pivot['должник'] / (children_pivot['должник'] + children_pivot['исправный плательщик']) * 100
children_pivot # выводим сводную таблицу

payer_type,должник,исправный плательщик,debtor_percent
children_availability,,,
детей нет,1063,13028,7.543822
дети есть,678,6685,9.208203


### Вывод

На 1000 клиентов без детей приходится 75 должников, а на 1000 клиентов с детьми - 92 должника, следовательно категория клиентов клиентов, у которых детей нет, является более надежной для банка.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Для ответа на вопрос также создадим сводную таблицу, только теперь данные сгруппируем по семейному положению - аргументом index функции pivot_table() будет являться столбец family_status. Также для каждой категории из данного столбца добавим столбец с процентом должников и отсортируем сводную таблицу по возрастанию.

In [44]:
# создаем сводную талбицу по семейному положению
family_status_pivot = df.pivot_table(index='family_status', columns='payer_type', values='purpose_type', aggfunc='count' )
# добавляем в сводную процент должников по каждой из категорий по семейному положению
family_status_pivot['debtor_percent'] =family_status_pivot['должник'] / (family_status_pivot['должник'] + family_status_pivot['исправный плательщик']) * 100
family_status_pivot.sort_values('debtor_percent') # выводим сводную таблицу

payer_type,должник,исправный плательщик,debtor_percent
family_status,,,
вдовец / вдова,63,896,6.569343
в разводе,85,1110,7.112971
женат / замужем,931,11408,7.545182
гражданский брак,388,3763,9.347145
Не женат / не замужем,274,2536,9.750890


### Вывод

Люди, когда-то бывшие или состоящие на текущий момент в браке, являются более благонадежными плательщиками, чем те, кто еще никогда не был скован этими узами. Предположительно, брак развивает чувство ответсвенности перед другим человеком, что также влияет на регулярность выплат по кредитам.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Здесь построим сводную таблицу по тому же принципу, только теперь столбцом, по которым будем группировать данные, является уровень дохода income_level. Также добавим процент должников для выявления зависимости и отсортируем по нему таблицу.

In [45]:
# создаем сводную талбицу по уровню дохода
income_level_pivot = df.pivot_table(index='income_level', columns='payer_type', values='purpose_type', aggfunc='count' )
# добавляем в сводную процент должников по каждой из категорий по уровню дохода
income_level_pivot['debtor_percent'] = income_level_pivot['должник'] / (income_level_pivot['должник'] + income_level_pivot['исправный плательщик']) * 100
mean_percent = income_level_pivot['должник'].sum() / (income_level_pivot['должник'].sum() + income_level_pivot['исправный плательщик'].sum()) * 100
print('Средний % должников по всей выборке:', mean_percent) #посчитали средний процент по всей выборке
income_level_pivot.sort_values('debtor_percent') # выводим сводную таблицу

Средний % должников по всей выборке: 8.115036822970076


payer_type,должник,исправный плательщик,debtor_percent
income_level,,,
выше бедности,1,43,2.272727
средний достаток,88,1146,7.131280
сверхбогатые,598,7463,7.418434
состоятельные,270,3011,8.229198
богатые,783,8045,8.869506
бедность,1,5,16.666667


### Вывод

В нашей выборке очень мало людей (всего 50 человек), имеющих уровень дохода "бедность" и "выше бедности", скорей всего банк в большинстве случаев отказывает в выдаче кредита данным категориям. В случае с категорией "бедность" этот отказ целесообразен, т.к. из 6 человек 1 стал должником. Процент должников по категории "выше бедности" предельно низкий по сравнению с остальными категорями и показывает, что банк теряет выручку, отказывая людям из этой категории в выдаче кредита. 

Из оставшихся категорий лучше всего отдают кредиты люди со средним достатком, но по сравнению с другими категориями (от состоятельных до сверхбогатых) в выборке таких людей значительно меньше. Это говорит о том, что банк не достаточно привлекает таких клиентов (из категории со средним достатком), хотя они более благонадежные на фоне с другими. По категории сверхбогатые клиентов достаточно много и % должников по ней находится на уровне ниже среднего, что говорит о том, что банк верно выстроил работу с данной нишей клиентов и никаких изменений по ней не требуется.

Состоятельные и богатые люди имеют близкий % должников. Т.к. этот % больше среднего, то при принятии решений по выдаче кредитов людям с данным уровнем дохода, банку следует проводить дополнительный анализ, исходя из ответов на другие поставленные вопросы.

- Как разные цели кредита влияют на его возврат в срок?

Проведем аналогичную процедуру как и при ответе на предыдущий вопрос, только теперь для аргумента index задаем столбец purpose_type.

In [46]:
# создаем сводную талбицу по целям
purpose_type_pivot = df.pivot_table(index='purpose_type', columns='payer_type', values='purpose', aggfunc='count' )
# добавляем в сводную процент должников по каждой из категорий по целям
purpose_type_pivot['debtor_percent'] = purpose_type_pivot['должник'] / (purpose_type_pivot['должник'] + purpose_type_pivot['исправный плательщик']) * 100
print('Средний % должников по всей выборке:', mean_percent) #еще раз покажем средний % должников
purpose_type_pivot.sort_values('debtor_percent') # выводим сводную таблицу

Средний % должников по всей выборке: 8.115036822970076


payer_type,должник,исправный плательщик,debtor_percent
purpose_type,,,
жилье,308,4152,6.905830
недвижимость,474,5877,7.463392
свадьба,186,2138,8.003442
образование,370,3643,9.220035
автомобиль,403,3903,9.359034


### Вывод

Самая выгодная категория целей для банка - это жилье, потому что люди боятся потерять "крышу над головой" и платят исправно. Чуть с меньшим процентом идет недвижимость, потому что под нее может попадать как жилая, так и не жилая недвижимость.
На средних позициях по должникам находится выдача денег на свадьбу.
Хуже всего возвращают деньги при получении кредита на образование или свадьбу, значит банку следует пристальней анализировать таких клиентов при принятии решения и руководствоваться дополнительными показателями из других категорий.

In [47]:
# создаем сводную талбицу по наличию детей
children_pivot = df.pivot_table(index='children_availability', columns='payer_type', values='purpose_type', aggfunc='count' )
# добавляем в сводную процент должников по каждой из категорий по детям
children_pivot['debtor_percent'] = (
    children_pivot['должник'] / (children_pivot['должник'] + children_pivot['исправный плательщик'])
     ).map('{:.2%}'.format)
children_pivot # выводим сводную таблицу

payer_type,должник,исправный плательщик,debtor_percent
children_availability,,,
детей нет,1063,13028,7.54%
дети есть,678,6685,9.21%


### Шаг 4. Общий вывод

По представленной банком выборке была проведена предобработка данных, включающая в себя: заполнение пропущенных значений, замену вещественного типа данных на целочисленный, обработку артефактов и удаление дубликатов. Из обработанной выборки были выделены основные категории для ответа на вопросы заказчика. По результам анализа выборки можно сказать следующее:

Вопрос 1. Среди клиентов без детей должников меньше, поэтому для получения большей прибыли с данной категории рекомендуем банку выделить доплнительные цели, которые будут наиболее пресущи людям без детей (например, путешествия) и запустить по ним дополнительную рекламу с предложением выгодных условий кредита на данные цели.

Вопрос 2. Среди клиентов, которые были или находятся в официальном браке, должников меньше на 2-3 %, поэтому при проверке анкет клиентов, которые не были в браке, банку следует обращать внимание на дополнительные факторы риска по ним.

Вопрос 3. Банку нужно больше привлекать клиентов с ежемесячным доходом от 22.5 до 33.5 т.р, потому что среди них предельно низкое количество должников. Также нужно больше привлекать людей со средним достатком и особое внимание обратить при рассмотрении анкет богатых и состоятельных людей.

Вопрос 4. Жилье и недвижимость - самые надежные категории в плане своевременных выплат. При выдаче кредитов на образование и автомобиль следует наиболее тщательным образом проверять платежоспособность клиента.